In [1]:
import pandas as pd

In [2]:
import warnings


warnings.filterwarnings('ignore')

In [3]:
pd.set_option('display.max_columns', 200)
pd.set_option('display.max_rows', 400)

In [5]:
statcast_2017 = pd.read_csv('../data/statcast_data/sc_2017.csv')
statcast_2018 = pd.read_csv('../data/statcast_data/sc_2018.csv')

In [18]:
statcast_2018

,index,pitch_type,game_date,release_speed,release_pos_x,release_pos_z,player_name,batter,pitcher,events,description,spin_dir,spin_rate_deprecated,break_angle_deprecated,break_length_deprecated,zone,des,game_type,stand,p_throws,home_team,away_team,type,hit_location,bb_type,balls,strikes,game_year,pfx_x,pfx_z,plate_x,plate_z,on_3b,on_2b,on_1b,outs_when_up,inning,inning_topbot,hc_x,hc_y,tfs_deprecated,tfs_zulu_deprecated,fielder_2,umpire,sv_id,vx0,vy0,vz0,ax,ay,az,sz_top,sz_bot,hit_distance_sc,launch_speed,launch_angle,effective_speed,release_spin_rate,release_extension,game_pk,pitcher.1,fielder_2.1,fielder_3,fielder_4,fielder_5,fielder_6,fielder_7,fielder_8,fielder_9,release_pos_y,estimated_ba_using_speedangle,estimated_woba_using_speedangle,woba_value,woba_denom,babip_value,iso_value,launch_speed_angle,at_bat_number,pitch_number,pitch_name,home_score,away_score,bat_score,fld_score,post_away_score,post_home_score,post_bat_score,post_fld_score,if_fielding_alignment,of_fielding_alignment
0,0,SL,2018-10-28,84.0,3.05,5.26,Chris Sale,592518.0,519242.0,strikeout,swinging_strike,NaN,NaN,NaN,NaN,13.0,Manny Machado strikes out swinging.,W,R,L,LAD,BOS,S,2.0,NaN,1.0,2.0,2018.0,-1.35,-0.26,-1.69,0.90,NaN,NaN,NaN,2.0,9.0,Bot,NaN,NaN,NaN,NaN,543877.0,NaN,181029_031720,-8.287278,-121.937872,-3.051222,-11.346649,26.649434,-34.287809,3.49,1.60,NaN,NaN,NaN,82.9,2681.0,5.9,563411.0,519242.0,543877.0,456665.0,571788.0,646240.0,593428.0,643217.0,598265.0,605141.0,54.56,NaN,NaN,0.0,1.0,0.0,0.0,NaN,65.0,4.0,Slider,1.0,5.0,1.0,5.0,5.0,1.0,1.0,5.0,Standard,Standard
1,1,FF,2018-10-28,95.3,3.17,5.50,Chris Sale,592518.0,519242.0,NaN,ball,NaN,NaN,NaN,NaN,12.0,NaN,W,R,L,LAD,BOS,B,NaN,NaN,0.0,2.0,2018.0,0.83,1.26,0.16,4.48,NaN,NaN,NaN,2.0,9.0,Bot,NaN,NaN,NaN,NaN,543877.0,NaN,181029_031659,-9.701178,-138.390708,0.212266,12.883696,33.319856,-16.484654,3.74,1.72,NaN,NaN,NaN,94.6,2431.0,6.2,563411.0,519242.0,543877.0,456665.0,571788.0,646240.0,593428.0,643217.0,598265.0,605141.0,54.35,NaN,NaN,NaN,NaN,NaN,NaN,NaN,65.0,3.0,4-Seam Fastball,1.0,5.0,1.0,5.0,5.0,1.0,1.0,5.0,Standard,Standard
2,2,FF,2018-10-28,96.4,3.07,5.54,Chris Sale,592518.0,519242.0,NaN,foul,NaN,NaN,NaN,NaN,3.0,NaN,W,R,L,LAD,BOS,S,NaN,NaN,0.0,1.0,2018.0,1.05,1.04,0.62,3.59,NaN,NaN,NaN,2.0,9.0,Bot,NaN,NaN,NaN,NaN,543877.0,NaN,181029_031631,-8.852806,-140.120015,-1.955200,15.725661,29.903912,-18.346800,3.49,1.60,NaN,NaN,NaN,96.3,2416.0,6.2,563411.0,519242.0,543877.0,456665.0,571788.0,646240.0,593428.0,643217.0,598265.0,605141.0,54.31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,65.0,2.0,4-Seam Fastball,1.0,5.0,1.0,5.0,5.0,1.0,1.0,5.0,Standard,Standard
3,3,CH,2018-10-28,86.6,3.10,4.98,Chris Sale,592518.0,519242.0,NaN,swinging_strike,NaN,NaN,NaN,NaN,13.0,NaN,W,R,L,LAD,BOS,S,NaN,NaN,0.0,0.0,2018.0,1.52,-0.02,-0.10,1.39,NaN,NaN,NaN,2.0,9.0,Bot,NaN,NaN,NaN,NaN,543877.0,NaN,181029_031609,-10.697003,-125.691745,-2.299318,18.251366,25.266016,-32.130579,3.49,1.60,NaN,NaN,NaN,86.3,2177.0,6.2,563411.0,519242.0,543877.0,456665.0,571788.0,646240.0,593428.0,643217.0,598265.0,605141.0,54.27,NaN,NaN,NaN,NaN,NaN,NaN,NaN,65.0,1.0,Changeup,1.0,5.0,1.0,5.0,5.0,1.0,1.0,5.0,Standard,Standard
4,4,SL,2018-10-28,78.5,3.31,5.16,Chris Sale,571771.0,519242.0,strikeout,swinging_strike,NaN,NaN,NaN,NaN,4.0,Enrique Hernandez strikes out swinging.,W,R,L,LAD,BOS,S,2.0,NaN,3.0,2.0,2018.0,-1.28,-0.25,-0.41,2.74,NaN,NaN,NaN,1.0,9.0,Bot,NaN,NaN,NaN,NaN,543877.0,NaN,181029_031528,-5.650976,-114.004641,2.039603,-9.806001,21.021107,-34.953400,3.32,1.51,NaN,NaN,NaN,77.2,2508.0,5.5,563411.0,519242.0,543877.0,456665.0,571788.0,646240.0,593428.0,643217.0,598265.0,605141.0,55.04,NaN,NaN,0.0,1.0,0.0,0.0,NaN,64.0,7.0,Slider,1.0,5.0,1.0,5.0,5.0,1.0,1.0,5.0,Standard,Standard
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,.

In [ ]:
# def previous_pitches(y)
#     pitch_sequence = {}
#     pitch_sequence[1] = 0
#     pitch_sequence[2] = 0
#     pitch_sequence[3] = 0
#     pitch_sequence[new_df.loc[(y+1), 'pitch']] += 1

#     #different_hitter_comp = (new_df.loc[i, 'at_bat_number'] != new_df.loc[y, 'at_bat_number'])
#     pitch_sequence[new_df.loc[(y+1), 'pitch']] += 1
    
#     for pitch in pitch_sequence:
#         new_df.loc[i, pitch] = pitch_sequence[pitch]

In [14]:
def df_by_year(df):
    #display(df)
#     if df.loc[0, 'pitcher'] == 519242.0:
    previous_year = statcast_2017
#     else:
#         previous_year = statcast_2018

    #display(previous_year_pitching)
    game_list = []
    
    #new_df.  is_copy = False
#     previous_year.is_copy = False

    pitch_columns = ['pitcher','batter', 'zone', 'pitch_number',
     'stand', 'p_throws', 'type',  'balls', 'strikes', 
      'on_3b', 'on_2b', 'on_1b','outs_when_up', 'inning', 
       'inning_topbot','woba_value', 'at_bat_number', 
       'pitch_name', 'bat_score', 'fld_score','if_fielding_alignment',
       'of_fielding_alignment','game_pk']

    new_df = df[pitch_columns]
    
#     new_df.is_copy = False

    fast = ['4-Seam Fastball', 'Fastball', 'Cutter', '2-Seam Fastball', 'Sinker']
    change = ['Changeup', 'Split-Finger', 'Knuckleball', 'Eephus', 'Forkball', 
              'Screwball']
    breaking = ['Curveball', 'Knuckle Curve', 'Slider']
    #slider = ['Slider']
    
    pitch_name = new_df['pitch_name']

    new_df['pitch'] = [1 if x in change else 2 if x in breaking else 3 for x in new_df[ 'pitch_name'].copy()]

    previous_year['pitch'] = [1 if x in change else 2 if x in breaking 
    else 3 for x in previous_year['pitch_name'].copy()]

    new_df[1] = 0
    new_df[2] = 0
    new_df[3] = 0

    pitch_recorder = pd.get_dummies(data=previous_year, columns=['pitch'], drop_first=True)[
    ['pitcher','pitch_2', 'pitch_3']].set_index('pitcher')

    pitcher_type = pitch_recorder.groupby('pitcher').mean()

    # Cannot predict for pitchers new to league, why I chose inner
    new_df = new_df.merge( pitcher_type, on='pitcher', how='inner')

    new_df['previous_pitch'] = new_df['pitch'].shift(-1)

    new_df['previous_zone'] = new_df['zone'].shift(-1)

    new_df['previous_type'] = new_df['type'].shift(-1)

    new_df['previous_hitter_woba'] = -1

    #new_df.reset_index(inplace=True, drop=True)

    new_df['extra_innings'] = 0

    new_df = pd.get_dummies(data=new_df, columns=['stand', 'p_throws'], drop_first=True)
    
    new_df = pd.get_dummies(data=new_df, columns=['if_fielding_alignment', 
                                                  'of_fielding_alignment'])
    
    new_df = new_df.drop(columns=['if_fielding_alignment_Strategic', 
                                'of_fielding_alignment_Strategic'])

    new_df['Starting_Pitcher'] = 0
    
    hit_gb = new_df.groupby(['game_pk','at_bat_number'])['woba_value'].mean()
#     display(hit_gb.loc[531699.0, 27.0])
    
    new_df['risp'] = 0
    
    #new_df.reset_index(drop=True, inplace=True)
    
    for i in new_df.index.to_list(): 
        if i == (len(new_df) - 1):
            new_df.loc[i, 'previous_pitch'] = 0
            new_df.loc[i, 'previous_zone'] = 0
            # Ensures I drop the values where balls are hit
            new_df.loc[i, 'previous_type'] = 'X'
            break
            
        #game_id = new_df.loc[i, 'game_pk']
        
        game_comp = (new_df.loc[i, 'game_pk'] == new_df.loc[i+1, 'game_pk'])
#         pitcher_comp = new_df.loc[i, 'pitcher'] == new_df.loc[i+1, 'pitcher']
#         half_inning_comp = new_df.loc[i, 'inning_topbot'] == new_df.loc[i+1, 'inning_topbot']
#         inning_comp = new_df.loc[i, 'inning'] == new_df.loc[i+1, 'inning']
        same_hitter_comp = (new_df.loc[i, 'at_bat_number'] == new_df.loc[(i+1), 'at_bat_number'])
        
        current_at_bat = new_df.loc[i, 'at_bat_number'] 
        current_game = new_df.loc[i, 'game_pk']
        
#         starting_home_pitcher = 0
#         starting_away_pitcher = 0
        
#         if (new_df.loc[i, 'inning'] == 1.0) & (new_df.loc[i,'inning_topbot']=='Bot'):
#             starting_away_pitcher = new_df.loc[i, 'pitcher']
#             game_id = new_df.loc[i, 'game_pk']
        
#         if (new_df.loc[i, 'inning'] == 1.0) & (new_df.loc[i, 'inning_topbot'] == 'Top'):
#             starting_home_pitcher = new_df.loc[i, 'pitcher']
            
#         if ((new_df.loc[i, 'pitcher'] == starting_home_pitcher) or (new_df.loc[i, 'pitcher'] == starting_away_pitcher)) & ( new_df.loc[i, 'game_pk'] == game_id):
#             new_df.loc[i, 'starting_pitcher'] = 1
#         else:
#             new_df.loc[i, 'starting_pitcher'] = 0

#         if new_df.loc[i, 'pitch_number'] == 0:
#             new_df.loc[i, 'previous_pitch'] = 0

        if not game_comp & same_hitter_comp:
            new_df.loc[i, 'previous_pitch'] = 0
            new_df.loc[i, 'previous_zone'] = 0
            # Ensures I drop the values where balls are hit
            new_df.loc[i, 'previous_type'] = 'X'
            #continue

        previous_at_bat = current_at_bat - 1
        
        #if inning_comp & game_comp & pitcher_comp & half_inning_comp & same_hitter_comp:
#         try:
#             if game_comp  & same_hitter_comp & (current_at_bat > 1):
#                 new_df.loc[i, 'previous_hitter_woba'] = hit_gb.loc[current_game, previous_at_bat] 
#         except:
#             continue
            
            #new_df.loc[i, 'previous_hitter_woba'] = new_df[new_df['at_bat_number'] == (current_at_bat - 1 ) & new_df['game_pk'] == current_game]['woba_value'].mean()
#             for y in range(i, len(new_df) -1):
                
#                 different_hitter_comp = (new_df.loc[i, 'at_bat_number'] != new_df.loc[y, 'at_bat_number'])

#                 if different_hitter_comp:
#                     new_df.loc[i, 'previous_hitter_woba'] = new_df.loc[y, 'woba_value']
#                     break

#                 elif game_comp & pitcher_comp & half_inning_comp:
#                     new_df.loc[i, 'previous_hitter_woba'] = new_df.loc[(i+1), 'woba_value']

                #pitch_sequence[new_df.loc[(y+1), 'pitch']] += 1

       
                
#             for pitch in pitch_sequence:
#                 new_df.loc[i, pitch] = pitch_sequence[pitch]
               
    
        
            
#         new_df['num_of_1_in_ab'] = pitch_squence[1]
                    
#         new_df['num_of_2_in_ab'] = pitch_sequence[2]
        
#         new_df['num_of_3_in_ab'] = pitch_sequence[3]
        
       
        
        if not game_comp:
            game_id = new_df.loc[(i+1), 'game_pk']
            sp_finder = new_df[new_df['game_pk'] == game_id]
            sp_finder_2 = sp_finder[sp_finder['inning'] == 1.0]
            sp_finder_home = sp_finder_2[sp_finder_2['inning_topbot'] == 'Top']['pitcher'].max()
            sp_finder_away = sp_finder_2[sp_finder_2['inning_topbot'] == 'Bot']['pitcher'].max()
            
        if i == 0:
            game_id = new_df.loc[(i), 'game_pk']
            sp_finder = new_df[new_df['game_pk'] == game_id]
            sp_finder_2 = sp_finder[sp_finder['inning'] == 1.0]
            sp_finder_home = sp_finder_2[sp_finder_2['inning_topbot'] == 'Top']['pitcher'].max()
            sp_finder_away = sp_finder_2[sp_finder_2['inning_topbot'] == 'Bot']['pitcher'].max()

        if (new_df.loc[i, 'pitcher'] == sp_finder_home) or (new_df.loc[i, 'pitcher'] == sp_finder_away):
            new_df.loc[i, 'Starting_Pitcher'] = 1
            
        if (new_df.loc[i, 'on_2b'] > 0) or (new_df.loc[i, 'on_3b'] > 0):
            new_df.loc[i, 'risp'] = 1
            
        if (new_df.loc[i, 'on_1b'] > 0):
            new_df.loc['on_1b'] = 1
        else: 
            new_df.loc['on_1b'] = 0
        
        
#         reverse = new_df[(new_df['at_bat_number'] == current_at_bat) & (new_df['game_pk'] == current_game)].sort_values('pitch_number',ascending=False)
#         reverse.reset_index(inplace=True)
#         display(reverse)
        pitch_sequence = {}
        pitch_sequence[1] = 0
        pitch_sequence[2] = 0
        pitch_sequence[3] = 0

        for y in range((i+1), len(new_df)-1):
            if new_df.loc[y, 'at_bat_number'] != current_at_bat:
                break
            pitch = new_df.loc[y, 'pitch']
            pitch_sequence[pitch] += 1
        
        for pitch in pitch_sequence:
            new_df.loc[i, pitch] = pitch_sequence[pitch]
        
        if (i % 1_000) == 0:
            print(i)
        
#         if i == 1000:
#             return new_df

                                
#     for i in new_df.index.to_list():
#         current_at_bat = new_df.loc[i, 'at_bat_number'] 
#         current_game = new_df.loc[i, 'game_pk']
#         if inning_comp & game_comp & pitcher_comp & half_inning_comp & same_hitter_comp:
            
#             reverse = new_df[(new_df['at_bat_number'] == current_at_bat) & (new_df['game_pk'] == current_game)].sort_values('pitch_number',ascending=False)
#             reverse.reset_index(inplace=True)
#             display(reverse)
#             for y in reverse.index.to_list():
#                 if (y+1) == len(reverse):
#                     break
#                 pitch = reverse.loc[(y+1), 'pitch']
#                 new_df.loc[i, pitch] += 1
#                 yeh
#     new_df.groupby(('game_pk','at_bat'))['pitch']
    
#         if new_df.loc[i, 'inning'] > 9:
#             new_df.loc[i,'inning'] = 1

#     for i in new_df.sort_values(by='innning', ascending=True):  
        
#         game_comp = new_df.loc[i, 'game_pk'] == new_df.loc[i+1, 'game_pk']
#         pitcher_comp = new_df.loc[i, 'pitcher'] == new_df.loc[i+1, 'pitcher']
#         half_inning_comp = new_df.loc[i, 'inning_topbot'] == new_df.loc[i+1, 'inning_topbot']
#         inning_comp = new_df.loc[i, 'inning'] == new_df.loc[i+1, 'inning']
#         same_hitter_comp = new_df.loc[i, 'at_bat_number'] == new_df.loc[(i+1), 'at_bat_number']
        
#         if (new_df.loc[i, 'inning'] == 1.0):
#             game_id = new_df.loc[i, 'game_pk']
        
#         if (new_df.loc[i, 'inning'] == 1.0) & (new_df.loc[i,'inning_topbot']=='Bot'):
#             starting_away_pitcher = new_df.loc[i, 'pitcher']
        
#         if (new_df.loc[i, 'inning'] == 1.0) & (new_df.loc[i, 'inning_topbot'] == 'Top'):
#             starting_home_pitcher = new_df.loc[i, 'pitcher']
            
#         if ((new_df.loc[i, 'pitcher'] == starting_home_pitcher) or (new_df.loc[i, 'pitcher'] == starting_away_pitcher)) & ( new_df.loc[i, 'game_pk'] == game_id):
#             new_df.loc[i, 'starting_pitcher'] = 1
#         else:
#             new_df.loc[i, 'starting_pitcher'] = 0


    new_df['previous_hitter_woba'].fillna(-1, inplace=True)

    # if new_df.loc[i, 'inning'] > 9:
    #   new_df.loc[i, 'extra_innings'] = 1
    #   new_df.loc[i, 'inning'] = 1

#     if i % 100_000 == 0:
#     print(i)
#     new_df['on_3b'].fillna(0, inplace=True)
#     new_df['on_2b'].fillna(0, inplace=True)
#     new_df['on_1b'] = [1 if x > 0 else 0 for x in new_df['on_1b']]
#     new_df['on_2b'] = [1 if x > 0 else 0 for x in new_df['on_2b']]
#     new_df['on_3b'] = [1 if x > 0 else 0 for x in new_df['on_3b']]  
#     new_df['risp_counter'] = new_df['on_2b'] + new_df['on_3b']
    
#     new_df['risp'] = [1 if x > 0 else 0 for x in (new_df['risp_counter'])]
    
    #new_df = new_df.drop(columns=['on_2b', 'on_3b', 'risp_counter'])

    #new_df['woba_value'] = new_df['woba_value'].fillna(0)

    new_df['run_diff'] = new_df['fld_score'] - new_df['bat_score']

    new_df = pd.get_dummies(data=new_df, columns=['inning_topbot'], drop_first=True)
    
    new_df = pd.get_dummies(data=new_df, columns=['previous_type'])
    new_df = new_df.drop(columns=['previous_type_X'])
    #                               'previous_type','previous_zone',
    #                               'previous_hitter_woba', 'previous_pitch'], 
    #                        drop_first=True)

#     new_df = pd.get_dummies(data=new_df, columns=['inning_topbot', 'previous_type'], 
#               drop_first=True)

    #new_df['extra_innings'] = [1 if x > 0 else 0 for x in sc_2018['inning_10.0']]

    sc_vc = previous_year['batter'].value_counts()
    statcast_vet_hitter = sc_vc[sc_vc > 500].index.to_list()

    train_merge = previous_year[previous_year['batter'].isin(statcast_vet_hitter)].groupby('batter')[['woba_value', 'launch_speed']].mean()

    new_df = new_df.merge(train_merge, on='batter')

    new_df = new_df.drop(columns=['at_bat_number', 'bat_score', 'fld_score', 'type',
                    'game_pk', 'pitcher', 
                    'pitch_name', 'woba_value_x', 'batter', 'pitch_number'])

# 'woba_value', batter
    return new_df




# new_df

In [15]:
o_1 = statcast_2018[statcast_2018['game_pk'] == 563411.0]

o_2 = statcast_2018[statcast_2018['game_pk'] == 563409.0]
game_ex = pd.concat([o_1, o_2])



In [16]:
new_df = df_by_year(statcast_2018)

0
1000


KeyboardInterrupt: 

In [11]:
new_df[500:700]

,pitcher,batter,zone,pitch_number,type,balls,strikes,on_3b,on_2b,on_1b,outs_when_up,inning,inning_topbot,woba_value,at_bat_number,pitch_name,bat_score,fld_score,game_pk,pitch,1,2,3,pitch_2,pitch_3,previous_pitch,previous_zone,previous_type,previous_hitter_woba,extra_innings,stand_R,p_throws_R,if_fielding_alignment_Infield shift,if_fielding_alignment_Standard,of_fielding_alignment_4th outfielder,of_fielding_alignment_Standard,Starting_Pitcher,risp
500,519242.0,643446.0,13.0,4.0,B,2.0,1.0,NaN,NaN,NaN,1.0,1.0,Top,NaN,2.0,4-Seam Fastball,0.0,0.0,531642.0,3,0,0,3,0.329542,0.487379,3.0,12.0,B,0.00,0,0,0,0,1,0,1,1,0
501,519242.0,643446.0,12.0,3.0,B,1.0,1.0,NaN,NaN,NaN,1.0,1.0,Top,NaN,2.0,4-Seam Fastball,0.0,0.0,531642.0,3,0,0,2,0.329542,0.487379,3.0,14.0,B,0.00,0,0,0,0,1,0,1,1,0
502,519242.0,643446.0,14.0,2.0,B,0.0,1.0,NaN,NaN,NaN,1.0,1.0,Top,NaN,2.0,2-Seam Fastball,0.0,0.0,531642.0,3,0,0,1,0.329542,0.487379,3.0,2.0,S,0.00,0,0,0,0,1,0,1,1,0
503,519242.0,643446.0,2.0,1.0,S,0.0,0.0,NaN,NaN,NaN,1.0,1.0,Top,NaN,2.0,4-Seam Fastball,0.0,0.0,531642.0,3,0,0,0,0.329542,0.487379,0.0,0.0,X,-1.00,0,0,0,0,1,0,1,1,0
504,519242.0,642708.0,12.0,1.0,X,0.0,0.0,NaN,NaN,NaN,0.0,1.0,Top,0.00,1.0,4-Seam Fastball,0.0,0.0,531642.0,3,0,0,0,0.329542,0.487379,0.0,0.0,X,-1.00,0,1,0,0,1,0,1,1,0
505,519242.0,545341.0,14.0,4.0,X,1.0,2.0,NaN,666971.0,434778.0,2.0,1.0,Top,0.00,5.0,Slider,0.0,0.0,531580.0,2,0,0,3,0.329542,0.487379,3.0,12.0,B,0.70,0,1,0,0,1,0,1,1,1
506,519242.0,545341.0,12.0,3.0,B,0.0,2.0,NaN,666971.0,434778.0,2.0,1.0,Top,NaN,5.0,4-Seam Fastball,0.0,0.0,531580.0,3,0,0,2,0.329542,0.487379,3.0,3.0,S,0.70,0,1,0,0,1,0,1,1,1
507,519242.0,545341.0,3.0,2.0,S,0.0,1.0,NaN,666971.0,434778.0,2.0,1.0,Top,NaN,5.0,4-Seam Fastball,0.0,0.0,531580.0,3,0,0,1,0.329542,0.487379,3.0,3.0,S,0.70,0,1,0,0,1,0,1,1,1
508,519242.0,545341.0,3.0,1.0,S,0.0,0.0,NaN,666971.0,434778.0,2.0,1.0,Top,NaN,5.0,4-Seam Fastball,0.0,0.0,531580.0,3,0,0,0,0.329542,0.487379,0.0,0.0,X,-1.00,0,1,0,0,1,0,1,1,1
509,519242.0,434778.0,13.0,5.0,B,3.0,1.0,NaN,666971.0,NaN,2.0,1.0,Top,0.70,4.0,Slider,0.0,0.0,531580.0,2,1,2,1,0.329542,0.487379,3.0,11.0,B,0.00,0,1,0,0,0,0,1,1,1


In [ ]:
hit_gb = statcast_2018.groupby(['game_pk','at_bat_number'])['woba_value'].mean()

In [ ]:
hit_gb.loc[531699.0, 27.0]

In [ ]:
statcast_2018[(statcast_2018['game_pk'] == 531699.0) & (statcast_2018['at_bat_number'] == 27.0)]

In [ ]:
new_df['risp'].value_counts()

In [ ]:
new_df